# MobileNet V2 Pre-Trained

In [ ]:
base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
base_model.trainable = False

In [ ]:
model_mobile = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(5, activation='softmax')
])

In [ ]:
model_mobile.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
initial_epochs = 10

callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ReduceLROnPlateau(patience=2)
]

In [ ]:
history = model_mobile.fit(
    train_generator,
    validation_data=val_generator,
    epochs=initial_epochs,
    callbacks=callbacks
)

## MobileNet V2 Pre-Trained - Unfreezing some layers

In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

In [ ]:
model_mobile_pre = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(5, activation='softmax')
])

In [ ]:
model_mobile_pre.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_mobile_pre.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
fine_tune_epochs = 10
history_finetune = model_mobile_pre.fit(
    train_generator,
    validation_data=val_generator,
    epochs=fine_tune_epochs,
    callbacks=callbacks
)

In [ ]:
model_mobile.export("mobilenet_saved_model")

In [ ]:
model_mobile_pre.export("mobilenet_pre_saved_model")

In [ ]:
model_mobile.save("mobilenet_model.keras")

In [ ]:
model_mobile_pre.save("mobilenet_pre_model.keras")

# TinyCNN

In [ ]:
img_size = 128
batch_size = 32

In [ ]:
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.2, patience=3, verbose=1)

In [ ]:
model_tinycnn = models.Sequential([
    layers.Conv2D(16, (3, 3), padding='same', activation='relu',
                  kernel_regularizer=regularizers.l2(1e-4), input_shape=(128, 128, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(32, (3, 3), padding='same', activation='relu',
                  kernel_regularizer=regularizers.l2(1e-4)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(1e-4)),
    layers.Dropout(0.5),
    layers.Dense(5, activation='softmax')
])

In [ ]:
model_tinycnn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
early_stop = EarlyStopping(
    patience=5,
    restore_best_weights=True,
    monitor='val_loss'
)

In [ ]:
reduce_lr = ReduceLROnPlateau(
    factor=0.2,
    patience=3,
    verbose=1
)

In [ ]:
history = model_tinycnn.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stop, reduce_lr]
)

In [ ]:
model_tinycnn.export("tinycnn_saved_model")

In [ ]:
model_tinycnn.save("tinycnn_model.keras")

# ResNet

## ResNet Common block

In [ ]:
def resnet_block(inputs, filters, kernel_size=3, stride=1):
    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same',
                      kernel_regularizer=regularizers.l2(1e-4))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, kernel_size, padding='same',
                      kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.BatchNormalization()(x)

    if stride != 1 or inputs.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, 1, strides=stride, padding='same')(inputs)
        shortcut = layers.BatchNormalization()(shortcut)
    else:
        shortcut = inputs

    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x

## ResNet8

In [ ]:
def build_resnet(input_shape=(128, 128, 3), num_classes=5, num_blocks=4):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(16, 3, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    filters = 16
    for i in range(num_blocks):
        stride = 2 if i % 2 == 0 and i != 0 else 1
        x = resnet_block(x, filters, stride=stride)
        filters *= 2 if i % 2 == 0 and i != 0 else 1

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

In [ ]:
model_resnet8 = build_resnet(num_blocks=4)

In [ ]:
model_resnet8.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

In [ ]:
history = model_resnet8.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stop, reduce_lr]
)

In [ ]:
model_resnet8.save("resnet8_model.keras")

In [ ]:
model_resnet8.export("resnet8_saved_model")

## ResNet10

In [ ]:
model_resnet10 = build_resnet(num_blocks=5)

In [ ]:
model_resnet10.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

In [ ]:
history = model_resnet8.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stop, reduce_lr]
)

In [ ]:
model_resnet10.save("resnet10_model.keras")

In [ ]:
model_resnet10.export("resnet10_saved_model")